## SLA Cleaner
The purpose of this notebook is to remove SLA assigned to the account based on the `Support Subsription Expiry Date` property.  
If subscription is expired, SLA is cleared with all related fields.  
Also, when customer is a part of dedicated group, some previleges are granted by default.  
Data is obtained from the custom view created on Zoho FrontEnd.  

In [ ]:
# setting notebook source
with open('./config_path.py') as f:
    exec(f.read())

In [ ]:
from config import secrets, settings
from zoho_api import *
import pandas as pd
import copy
from dateutil import parser
from datetime import date, time

In [ ]:
backup_data = {}
response_data = {}

In [ ]:
refresh_token()

In [ ]:
SLAs = {}
SLAs['SupportME Premium'] = '10977000006774075'
SLAs['SupportME Comfort'] = '10977000006804083'
SLAs['SupportME ComfortPlus'] = '10977000006804083'
SLAs['SupportME Standard'] = '10977000006804131'
SLAs[None] = ''
SLAs

In [ ]:
SupportHours = {}
SupportHours['SupportME Premium'] = 204
SupportHours['SupportME Comfort'] = 96
SupportHours['SupportME ComfortPlus'] = 8
SupportHours['SupportME Standard'] = 36
SupportHours[None] = None
SupportHours

## Tests

In [ ]:
read_account('10977000000586229')

## Application

In [ ]:
field_list = ['id'
              , 'accountName'
              , 'cf_account_id'
              , 'website'
              , 'cf_account_id'
              , 'cf_support_package'
              , 'cf_support_subsription_expiry_date'
              , 'associatedSLAIds'
              , 'cf_max_support_time'
              , 'cf_picklist_1'  # Customer type
              , 'cf_loyality_program'
             ]

In [ ]:
accounts = list_accounts( view="SI Elite customers", field_list=field_list)

In [ ]:
len(accounts)

In [ ]:
accounts_updated = copy.deepcopy(accounts)
today = date.today()
modifications = {}
expired_accounts = {}
for account in accounts_updated:
    support_package = ''
    # validation of expiry date
    if account['cf']['cf_support_subsription_expiry_date'] is not None:
        expiry_date = parser.parse(account['cf']['cf_support_subsription_expiry_date']).date()
    else:
        expiry_date = today
        
    # customer is distributor     
    if account['cf']['cf_picklist_1'] == 'Distributor': 
        support_package = 'SupportME Premium'
        account['cf']['cf_support_subsription_expiry_date'] = None # have no expiration date for support
    
    # customer is FDI and no extra support package is used
    elif account['cf']['cf_picklist_1'] == 'FDI' and account['cf']['cf_support_package'] is None: 
        support_package = 'SupportME Standard'
        account['cf']['cf_support_subsription_expiry_date'] = None # have no expiration date for support
    
    # package have expired
    elif today > expiry_date : 
        expired_accounts[account['cf']['cf_account_id'] ] = copy.deepcopy(account)
        support_package = None
        account['cf']['cf_support_subsription_expiry_date'] = support_package
    
    # package have faulty name and shall be removed
    elif account['cf']['cf_support_package'] not in SupportHours.keys(): 
        support_package = None
        account['cf']['cf_support_subsription_expiry_date'] = support_package

    # if any modification have been applied on data, prepare setup
    if support_package != '':   
        zoho_id = account.pop('id')
        account.pop('accountName')
        account.pop('associatedSLAIds')
        account['cf'].pop('cf_account_id')
        account['cf'].pop('cf_picklist_1')
        account['cf']['cf_support_package'] = support_package
        account['cf']['cf_max_support_time'] = SupportHours[support_package]
        account['website'] = support_package
        modifications[zoho_id] = account

In [ ]:
len(expired_accounts)

In [ ]:
processedIds = set()
missingIds = set()

In [ ]:
refresh_token()

for index, record in enumerate(modifications.items()):
    if record[0] in processedIds:
        continue
    account = backup_updated_fields(record[0], record[1], backup_data, response_data)
    if account is None:
        missingIds.add(record[0])
    else:
        try:
            account, response = update_account(record[0], record[1])
            if account is None or response.status_code != 200:
                missingIds.add(record[0])
                print(f"\r{index + 1}) Error for updating account: {record[0]}. \t\t")
            else:
                support_package = record[1]['cf']['cf_support_package']
                print(f"{index + 1}) Data for {record[0]} have been updated.\t", end='')
                apply_sla(record[0], SLAs[support_package])
                processedIds.add(record[0])
        except Exception as e:   
            missingIds.add(record[0])
            print(f"\r{index + 1}) Error for executing script for account: {record[0]}: {e}")
print("Updating process have been finished")

In [ ]:
updated_fields = record[1]
response_filename = "response_" +str(today) + ".json"
backup_filename = "backup_" +str(today) + ".json"
save_response_to_file(response_data = response_data,  filename=response_filename, updated_fields=updated_fields)
save_backup_to_file(backup_data, filename=backup_filename, updated_fields=updated_fields)